First, packages are imported.

In [36]:
#   import packages
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
import networkx as nx
import pickle
import functools

from ema_workbench import (
    Model,
    Policy,
    Scenario,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    util,
    ScalarOutcome,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import ArchiveLogger, EpsilonProgress
from ema_workbench.em_framework import parameters_from_csv
from ema_workbench.em_framework.evaluators import BaseEvaluator
from ema_workbench.analysis import parcoords
from ema_workbench.analysis import prim
from ema_workbench import Samplers

Load results from Uncertainty Analysis

In [37]:
uncertainty_results = pd.read_pickle(r'../generated_datasets/policy_uncertainty_provinces.pkl')
experiments_and_results = pd.read_pickle(r'../generated_datasets/policy_uncertainty_experiments_results.pkl')
aggregated_outcomes = pd.read_pickle(r'../generated_datasets/policy_uncertainty_aggregated.pkl')
uncertainty_results_run = pd.read_pickle(r'../generated_datasets/policy_uncertainty_test.pkl')

# Calculate Robustness Metrics

First, the time data is combined into a single value, and saved into a DataFrame.

In [38]:
experiments, outcomes = uncertainty_results_run

results_df = pd.DataFrame()
for k in outcomes:
    test_list = []
    for i in range(len(experiments)):
        t = outcomes[k][i].sum()
        test_list.append(t)
    results_df[k]=test_list

Then, we calculate the cost of death for all death columns, making it able to easily construct a total cost.

In [39]:
death_cost_per_person = 6_300_000

# calculate the cost of death so every column has the same unit (€)
results_df["A.1_Deaths_Cost"]=results_df["A.1_Expected Number of Deaths"]*death_cost_per_person
results_df["A.2_Deaths_Cost"]=results_df["A.2_Expected Number of Deaths"]*death_cost_per_person
results_df["A.3_Deaths_Cost"]=results_df["A.3_Expected Number of Deaths"]*death_cost_per_person
results_df["A.4_Deaths_Cost"]=results_df["A.4_Expected Number of Deaths"]*death_cost_per_person
results_df["A.5_Deaths_Cost"]=results_df["A.5_Expected Number of Deaths"]*death_cost_per_person

to_drop = ["A.1_Expected Number of Deaths","A.2_Expected Number of Deaths","A.3_Expected Number of Deaths","A.4_Expected Number of Deaths","A.5_Expected Number of Deaths"]
results_df.drop(to_drop, axis=1, inplace=True)

### Gelderland - Overijssel Split

A function is created to sort the data into two columns: Gelderland and Overijssel.

In [40]:
def combine_columns_province(dataframe, name):
    combined_columns_gelderland = []
    combined_columns_overijssel = []
    for x in dataframe.columns:
        if name in x:
            if x.startswith('A.1') or x.startswith('A.2') or x.startswith('A.3'):
                combined_columns_gelderland.append(x)
            if x.startswith('A.4') or x.startswith('A.5'):
                combined_columns_overijssel.append(x)


    dataframe['Total ' + str(name) + ' Gelderland'] = dataframe[combined_columns_gelderland].sum(axis=1)
    dataframe['Total ' + str(name) + ' Overijssel'] = dataframe[combined_columns_overijssel].sum(axis=1)


A new DataFrame is then created with the province-sorted data.

In [41]:
provinces = results_df.copy()

combine_columns_province(provinces, 'Expected Annual Damage')
combine_columns_province(provinces, 'Dike Investment Costs')
combine_columns_province(provinces, 'Deaths_Cost')

The excess columns are then deleted, and columns are created with total cost and total cost for the province of Overijssel.

In [42]:
aggregated_outcomes_provinces = pd.concat([experiments[['scenario','policy']],provinces], axis=1).iloc[:,2:]

for x in aggregated_outcomes_provinces.columns:

        if x.startswith('A.1') or x.startswith('A.2') or x.startswith('A.3') or x.startswith('A.4') or x.startswith('A.5'):
            aggregated_outcomes_provinces.drop(x, axis=1, inplace=True)
        else:
            pass

aggregated_outcomes_provinces['total_cost[€]']= aggregated_outcomes_provinces.sum(axis=1)
overijssel_sum = ['RfR Total Costs','Expected Evacuation Costs','Total Expected Annual Damage Overijssel','Total Dike Investment Costs Overijssel','Total Deaths_Cost Overijssel']
aggregated_outcomes_provinces['total_cost[€]_overijssel']= aggregated_outcomes_provinces.loc[:,overijssel_sum].sum(axis=1)

aggregated_outcomes_provinces

,RfR Total Costs,Expected Evacuation Costs,Total Expected Annual Damage Gelderland,Total Expected Annual Damage Overijssel,Total Dike Investment Costs Gelderland,Total Dike Investment Costs Overijssel,Total Deaths_Cost Gelderland,Total Deaths_Cost Overijssel,total_cost[€],total_cost[€]_overijssel
0,121200000.0,7440.699768,2.598958e+08,0.000000e+00,4.261252e+08,8.364938e+07,322971.446503,0.000000,8.912008e+08,2.048568e+08
1,121200000.0,529.613015,2.007740e+07,0.000000e+00,4.261252e+08,8.364938e+07,25938.556696,0.000000,6.510785e+08,2.048499e+08
2,121200000.0,1131.697699,2.169477e+07,0.000000e+00,4.261252e+08,8.364938e+07,54763.469958,0.000000,6.527253e+08,2.048505e+08
3,121200000.0,3997.055938,1.138822e+08,0.000000e+00,4.261252e+08,8.364938e+07,181533.542707,0.000000,7.450423e+08,2.048534e+08
4,121200000.0,12299.407927,2.680297e+08,0.000000e+00,4.261252e+08,8.364938e+07,527180.556318,0.000000,8.995438e+08,2.048617e+08
...,...,...,...,...,...,...,...,...,...,...
66995,0.0,5617.388654,2.618213e+07,4.928776e+07,4.000580e+08,8.641652e+07,49677.442362,44187.468244,5.620439e+08,1.357541e+08
66996,0.0,2236.872345,2.509358e+07,7.452276e+06,4.000580e+08,8.641652e+07,50151.312206,4283.788780,5.190771e+08,9.387531e+07
66997,0.0,1140.688872,4.610411e+07,1.455608e+06,4.000580e+08,8.641652e+07,35518.629332,874.937299,5.340718e+08,8.787414e+07
66998,0.0,438.029693,4.829867e+06,3.099803e+06,4.000580e+08,8.641652e+07,8316.725584,1432.709180,4.944144e+08,8.951819e+07


Lastly, we add the policy number.

In [43]:
aggregated_outcomes_provinces_policy = aggregated_outcomes_provinces.copy()
aggregated_outcomes_provinces_policy['policy'] = experiments.iloc[:,-2]
aggregated_outcomes_provinces_policy

,RfR Total Costs,Expected Evacuation Costs,Total Expected Annual Damage Gelderland,Total Expected Annual Damage Overijssel,Total Dike Investment Costs Gelderland,Total Dike Investment Costs Overijssel,Total Deaths_Cost Gelderland,Total Deaths_Cost Overijssel,total_cost[€],total_cost[€]_overijssel,policy
0,121200000.0,7440.699768,2.598958e+08,0.000000e+00,4.261252e+08,8.364938e+07,322971.446503,0.000000,8.912008e+08,2.048568e+08,0
1,121200000.0,529.613015,2.007740e+07,0.000000e+00,4.261252e+08,8.364938e+07,25938.556696,0.000000,6.510785e+08,2.048499e+08,0
2,121200000.0,1131.697699,2.169477e+07,0.000000e+00,4.261252e+08,8.364938e+07,54763.469958,0.000000,6.527253e+08,2.048505e+08,0
3,121200000.0,3997.055938,1.138822e+08,0.000000e+00,4.261252e+08,8.364938e+07,181533.542707,0.000000,7.450423e+08,2.048534e+08,0
4,121200000.0,12299.407927,2.680297e+08,0.000000e+00,4.261252e+08,8.364938e+07,527180.556318,0.000000,8.995438e+08,2.048617e+08,0
...,...,...,...,...,...,...,...,...,...,...,...
66995,0.0,5617.388654,2.618213e+07,4.928776e+07,4.000580e+08,8.641652e+07,49677.442362,44187.468244,5.620439e+08,1.357541e+08,66
66996,0.0,2236.872345,2.509358e+07,7.452276e+06,4.000580e+08,8.641652e+07,50151.312206,4283.788780,5.190771e+08,9.387531e+07,66
66997,0.0,1140.688872,4.610411e+07,1.455608e+06,4.000580e+08,8.641652e+07,35518.629332,874.937299,5.340718e+08,8.787414e+07,66
66998,0.0,438.029693,4.829867e+06,3.099803e+06,4.000580e+08,8.641652e+07,8316.725584,1432.709180,4.944144e+08,8.951819e+07,66


### Generate outcome optimal for Overijssel

The best (cheapest) policies are selected. For this, we need the robustness metric again: Mean Square Deviation.

In [104]:
grouping = aggregated_outcomes_provinces_policy.groupby(['policy']).agg({'total_cost[€]_overijssel':['mean', 'std']})
grouping[('total_cost[€]_overijssel','mean square deviation')] = np.square(grouping[('total_cost[€]_overijssel', 'mean')]) + np.square(grouping[('total_cost[€]_overijssel', 'std')])

In [99]:
n_scenarios_of_interest = 5

results_of_interest = grouping.sort_values(by=('total_cost[€]_overijssel','mean square deviation'), ascending=True).head(n_scenarios_of_interest)

with open(r'..\generated_datasets\final_policies_overijssel.pkl','wb') as pickle_file:
    pickle.dump(results_of_interest, pickle_file)

results_of_interest

total_cost[€]_overijssel                                    
                           mean           std mean square deviation
policy                                                             
7                  7.251303e+07  2.398231e+07          5.833290e+15
6                  7.715460e+07  2.411583e+07          6.534405e+15
4                  7.924735e+07  2.243784e+07          6.783599e+15
22                 9.973077e+07  3.087734e+07          1.089964e+16
9                  1.039058e+08  2.849956e+07          1.160864e+16

### Generate outcome optimal for Gelderland and Overijssel combined.

In [103]:
grouping = aggregated_outcomes_provinces_policy.groupby(['policy']).agg({'total_cost[€]':['mean', 'std']})
grouping[('total_cost[€]','mean square deviation')] = np.square(grouping[('total_cost[€]', 'mean')]) + np.square(grouping[('total_cost[€]', 'std')])

In [102]:
n_scenarios_of_interest = 5

results_of_interest = grouping.sort_values(by=('total_cost[€]','mean square deviation'), ascending=True).head(n_scenarios_of_interest)

with open(r'..\generated_datasets\final_policies.pkl','wb') as pickle_file:
    pickle.dump(results_of_interest, pickle_file)

results_of_interest

total_cost[€]                                    
                mean           std mean square deviation
policy                                                  
42      6.735707e+08  1.441888e+08          4.744879e+17
9       6.355444e+08  2.908834e+08          4.885299e+17
41      6.693379e+08  2.284622e+08          5.002081e+17
66      6.436564e+08  3.294160e+08          5.228085e+17
23      7.303488e+08  7.537737e+07          5.390911e+17

### PRIM

In [46]:
x = experiments_and_results.iloc[:, :19]

for var in aggregated_outcomes.iloc[:, 2:].columns:
    var_path = str('../visualizations/Scenario_Discovery/policy_' + var + '_PRIM.png')
    var2_path = str('../visualizations/Scenario_Discovery/policy_' + var + '_PRIM_inspect.png')

    y_total_damage = aggregated_outcomes[var].values
    y_total_damage = y_total_damage > np.percentile(y_total_damage, 75)

    prim_alg_DAM = prim.Prim(x,
                             y_total_damage,
                             threshold=0.70,
                             peel_alpha=0.05, )  # deze variabelen nog tweaken
    box_DAM = prim_alg_DAM.find_box()

    img = box_DAM.show_tradeoff()
    plt.title(var)
    fig = img.get_figure()
    fig.savefig(var_path, bbox_inches='tight')
    fig.clf()

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>